<a href="https://colab.research.google.com/github/Souzajaum/land_uses_analysis/blob/main/land_uses_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap.core as geemap
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [3]:
ee.Authenticate()
ee.Initialize(project='land-use-analysis')

In [16]:
# ----------------------------------------
# 1. DEFINIÇÃO DA ÁREA DE ESTUDO
# ----------------------------------------
santa_maria = ee.Geometry.Rectangle([-53.85, -29.75, -53.45, -29.60])

# ----------------------------------------
# 2. COLETA E PRÉ-PROCESSAMENTO DE DADOS
# ----------------------------------------
# Filtrar imagens Sentinel-2 (bandas de 10m e 20m)
sentinel = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(santa_maria)
            .filterDate('2023-01-01', '2023-12-31')
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))

# Criar mosaico mediano e remover bandas desnecessárias
sentinel_median = sentinel.median().select(['B2', 'B3', 'B4', 'B8', 'B11'])

# Calcular índices espectrais
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndvi).addBands(ndwi)

image = add_indices(sentinel_median.clip(santa_maria))

In [22]:
# ----------------------------------------
# 3. AMOSTRAS DE TREINAMENTO
# ----------------------------------------
# Definir classes e paleta de cores profissional
class_info = {
    'names': ['Urbano', 'Agricultura', 'Floresta', 'Água', 'Solo Exposto'],
    'colors': ['#FF0000', '#FFFF00', '#00FF00', '#0000FF', '#A9A9A9'],
    'codes': [0, 1, 2, 3, 4]
}

# Amostras de treinamento (FALTA DEFINIR AS AMOSTRAS REAIS)
rois = ee.FeatureCollection([
    # Urbano (10 amostras)
    ee.Feature(ee.Geometry.Point([-53.70, -29.68]), {'class': 0}),
    ee.Feature(ee.Geometry.Point([-53.705, -29.683]), {'class': 0}),

    # Agricultura (10 amostras)
    ee.Feature(ee.Geometry.Point([-53.65, -29.72]), {'class': 1}),
    ee.Feature(ee.Geometry.Point([-53.655, -29.715]), {'class': 1}),

    # Floresta (10 amostras)
    ee.Feature(ee.Geometry.Point([-53.80, -29.73]), {'class': 2}),
    ee.Feature(ee.Geometry.Point([-53.795, -29.735]), {'class': 2}),

    # Água (10 amostras)
    ee.Feature(ee.Geometry.Point([-53.55, -29.65]), {'class': 3}),
    ee.Feature(ee.Geometry.Point([-53.555, -29.655]), {'class': 3}),

    # Solo Exposto (10 amostras)
    ee.Feature(ee.Geometry.Point([-53.58, -29.68]), {'class': 4}),
    ee.Feature(ee.Geometry.Point([-53.585, -29.685]), {'class': 4}),
])

# Extrair valores espectrais das amostras
training_data = image.sampleRegions(
    collection=rois,
    properties=['class'],
    scale=10,
    tileScale=8
)

In [18]:
# ----------------------------------------
# 4. CLASSIFICAÇÃO E VALIDAÇÃO
# ----------------------------------------
# Dividir em treino (70%) e teste (30%)
training_partition = training_data.randomColumn()
train_set = training_partition.filter(ee.Filter.lt('random', 0.7))
test_set = training_partition.filter(ee.Filter.gte('random', 0.7))

# Treinar Random Forest
classifier = ee.Classifier.smileRandomForest(100).train(
    features=train_set,
    classProperty='class',
    inputProperties=image.bandNames()
)

# Classificar a imagem
classified_image = image.classify(classifier)

# Validação com dados de teste
validation = test_set.classify(classifier)
confusion_matrix = validation.errorMatrix('class', 'classification')
print(f'Acurácia Global: {confusion_matrix.accuracy().getInfo():.2%}')
print(f'Índice Kappa: {confusion_matrix.kappa().getInfo():.2%}')


Acurácia Global: 33.33%
Índice Kappa: 0.00%


In [21]:
# ----------------------------------------
# 5. VISUALIZAÇÃO
# ----------------------------------------
Map = geemap.Map()
Map.centerObject(santa_maria, 10)

# Adicionar camadas
Map.addLayer(
    classified_image,
    {'min': 0, 'max': 4, 'palette': class_info['colors']},
    'Classificação de Uso do Solo'
)

Map.addLayer(
    sentinel_median,
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.2},
    'Imagem RGB Sentinel-2'
)



# Estilo adicional
Map.addLayer(santa_maria, {'color': 'white'}, 'Área de Estudo')
Map.addLayer(rois.style(**{'color': 'white', 'pointSize': 8}), {}, 'Amostras de Treinamento')

Map

Map(center=[-29.67513150320705, -53.65], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…